In [ ]:
import os
from osgeo import ogr, osr

from MalardClient.MalardClient import MalardClient
from MalardClient.DataSet import DataSet

client = MalardClient("MALARD-PROD")


proj4 = client.getProjection(DataSet( "test", "swath_c_GrIS_OIB", "greenland"  )).proj4
print(proj4)


daShapefile = "/data/eagle/project-cryo-tempo/data/masks/greenland/NoPeripheryMerged/GRE_Basins_IMBIE2_v1.3_NoPeriph.shp"
#daShapefile = "/data/puma1/scratch/mtngla/chucach/Chucach-polygon.shp"

#kmz file
#daShapeFile = "/data/mouse1/team/jon/Anarctica/Coastline_Antarctica_v02.shp"


# Save extent to a new Shapefile
outShapefile = "icesheet_noperiph.shp"

driver = ogr.GetDriverByName('ESRI Shapefile')

dataSource = driver.Open(daShapefile) # 0 means read-only. 1 means writeable.
feature = None

layer = dataSource.GetLayer(0)

for i in range(0, layer.GetFeatureCount()):
    f = layer.GetFeature(i)
    
    print(f.GetField(0))
#    if str(f.GetField(1)).startswith("LRM"):
#        print( f.GetField(1))
    
#    if f.GetField(1) == "SARIN over Greenland Coast":    #SARIN over Greenland Coast, LRM over Antarctica ice sheet, LRM over Greenland ice sheet,  
#        feature = f
#        print(f.GetField(1))
#    if f.GetField(1) == "SARIN over Greenland Coast":
#        print(f.GetGeometryRef().GetEnvelope())


#greenlandExtent = (-81.265, -6.8349, 59.1676, 84.2163)

        



In [ ]:


sourceprj = layer.GetSpatialRef()

srs = osr.SpatialReference()
srs.ImportFromProj4(proj4)

targetprj = srs
transform = osr.CoordinateTransformation(sourceprj, targetprj)
        
        
outDriver = ogr.GetDriverByName("ESRI Shapefile")

# Remove output shapefile if it already exists
if os.path.exists(outShapefile):
    outDriver.DeleteDataSource(outShapefile)

# Create the output shapefile
outDataSource = outDriver.CreateDataSource(outShapefile)
outlayer = outDataSource.CreateLayer("icesheet_noperiph", targetprj,geom_type=ogr.wkbPolygon)

# Add an ID field
idField = ogr.FieldDefn("id", ogr.OFTInteger)
outlayer.CreateField(idField)

# Create the feature and set values
for i in range(0, layer.GetFeatureCount()):
    feature = layer.GetFeature(i)

    transformed = feature.GetGeometryRef()
    transformed.Transform(transform)
    
    def replacePoint(wkt, x, y ):
        strToReplace = "{} {}".format(x, y)
        strReplacement = "{} {}".format(x, y + 100)
        
        return wkt.replace(strToReplace, strReplacement)
       
    gb = transformed.Buffer(0)
    
    geom = ogr.CreateGeometryFromWkb(gb.ExportToWkb()) 
    
    print(geom.IsValid())
    
    defn = outlayer.GetLayerDefn()
    feat = ogr.Feature(defn)
    feat.SetGeometry(geom)
    outlayer.CreateFeature(feat)
    

# Save and close DataSource
inDataSource = None
outDataSource = None






#Warning 1: Self-intersection at or near point 182103.98757600426 -2898036.0123973158
#Warning 1: Self-intersection at or near point 540934.02211438434 -1894356.1721276832
